In [1]:
pip install tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

#0. Initial Setting

In [3]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([1.5,1,-1.9,-1.8,1]) #RS, FC1,FC2, EC

trans_true = [0.9,0.3,0.8,0.85]
obser_true = [0.1,0.4,0.2,0.3]
theta23 = trans_true + obser_true

agent = 2
za_dim =(agent+1)*(2**agent)
za_old = np.arange(za_dim,dtype = int)

TimePeriod= 100
SamplePath = 4000
SubSample = 10

z_old = np.arange(agent+1,dtype=int) #observation
a1_old = np.arange(2,dtype = int)
a2_old = np.arange(2,dtype = int)

num_discrete = 100
x_old = np.linspace(0,1,num_discrete)

In [4]:
#print(za_old//(agent+1))
print((za_old//(agent+1))//2) #a1
print((za_old//(agent+1))%2)  #a2
print(za_old%(agent+1))       #z

np.hstack([np.hstack([a1_old[i] *2 + a2_old for i in range(2)])[j] *3+z_old for j in range(4)])
print((1*2+1)*3+2)

[0 0 0 0 0 0 1 1 1 1 1 1]
[0 0 0 1 1 1 0 0 0 1 1 1]
[0 1 2 0 1 2 0 1 2 0 1 2]
11


#1. Generate Dynamic Part and Policy Part

In [8]:
def Dynamic(theta23):

  trans_val = theta23[0:4]#[0.9,0.3,0.8,0.85]
  obser_val = theta23[4:8]#[0.1,0.4,0.2,0.3]
  trans = np.zeros([2,2,4]) #s_t,s_{t+1},a1 a2
  trans[0,0,0] = trans[1,1,3] = trans_val[0]
  trans[0,1,0] = trans[1,0,3] = 1 - trans_val[0]
  trans[1,0,0] = trans[0,1,3] = trans_val[1]
  trans[1,1,0] = trans[0,0,3] = 1 - trans_val[1]
  trans[0,0,1] = trans[1,1,1] = trans_val[2]
  trans[1,0,1] = trans[0,1,1] = 1 - trans_val[2]
  trans[0,0,2] = trans[1,1,2] = trans_val[3]
  trans[1,0,2] = trans[0,1,2] = 1 - trans_val[3]
  
  observe1 = np.zeros([2,3,3]) #s_{t+1},z_t,z_{t+1}
  observe2 = np.zeros([2,3,3]) #s_{t+1},z_t,z_{t+1}

  for s in range(2):
    observe1[s,0,1] = observe1[s,1,0] = observe1[s,1,2]= observe1[s,2,1] = obser_val[s]
    observe1[s,0,0] = 1 - observe1[s,0,1] -observe1[s,0,2]
    observe1[s,1,1] = 1 - observe1[s,1,0] -observe1[s,1,2]
    observe1[s,2,2] = 1 - observe1[s,2,0] - observe1[s,2,1]

  for s in range(2):
    observe2[s,0,1] = observe2[s,1,0] = observe2[s,1,2]= observe2[s,2,1] = obser_val[s+2]
    observe2[s,0,0] = 1 - observe2[s,0,1] -observe2[s,0,2]
    observe2[s,1,1] = 1 - observe2[s,1,0] -observe2[s,1,2]
    observe2[s,2,2] = 1 - observe2[s,2,0] - observe2[s,2,1]

  return trans,observe1,observe2
#print(Dynamic(theta23))

def Reward(theta1,ag_i): # 3*2*2, 2,2 #[1.5,1,-1.9,-1.8,1]
  rw = np.zeros([2,(agent+1)*(2**agent),2,2])#st,z*a_{t-1},a_{t,1},a_{t,2}
  for s in range(2):
    if ag_i==1:
      rw[s,:,1,0] = (za_old%(agent+1)) * theta1[s]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2))
      rw[s,:,1,1] = (za_old%(agent+1)) * theta1[s]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2))
    elif ag_i==2:
      rw[s,:,0,1] = (za_old%(agent+1)) * theta1[s]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2))
      rw[s,:,1,1] = (za_old%(agent+1)) * theta1[s]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2))    

  return rw
#print(Reward(theta1,2))

#trans,observe1, observe2 = Dynamic(theta23)  #s_t,s_{t+1},a1 a2 #s_{t+1},z_t,z_{t+1}#s_{t+1},z_t,z_{t+1}

def SigmaLambda(za_old,z_old,x1_old,x2_old,trans,observe1,observe2,a1_old,a2_old,num_discrete=num_discrete,T = None): #(xt1 or xt2),z_t,z_{t+1},a_t
  if T==None: #generate function for Q function iteration
      x1_new = np.zeros([num_discrete,3,3,4])
      x2_new = np.zeros([num_discrete,3,3,4])

      sigma1 = np.zeros([num_discrete,3,3,4])
      sigma2 = np.zeros([num_discrete,3,3,4])

      x1_old = np.vstack([x1_old,1-x1_old]).T
      x2_old = np.vstack([x2_old,1-x2_old]).T

      for z_new in z_old:
        for a1 in a1_old:
          for a2 in a2_old:
            a_old = a1*2+a2
            #for za in za_old:#range(2):
            #za = za_old%(agent+1)
            x1_new_tep = x1_old.dot(np.reshape(trans[:,0,a_old],[-1,1]).dot(observe1[0,:,z_new].reshape([1,-1])))
            sigma1[:,:,z_new,a_old] = x1_new_tep + x1_old.dot(np.reshape(trans[:,1,a_old],[-1,1]).dot(observe1[1,:,z_new].reshape([1,-1])))
            sigma1_0 = np.where(sigma1[:,:,z_new,a_old]!=0)
            x1_new[:,:,z_new,a_old][sigma1_0] = x1_new_tep[sigma1_0] /sigma1[:,:,z_new,a_old][sigma1_0]

            x2_new_tep = x2_old.dot(np.reshape(trans[:,0,a_old],[-1,1]).dot(observe2[0,:,z_new].reshape([1,-1])))
            sigma2[:,:,z_new,a_old] = x2_new_tep + x2_old.dot(np.reshape(trans[:,1,a_old],[-1,1]).dot(observe2[1,:,z_new].reshape([1,-1])))
            sigma2_0 = np.where(sigma2[:,:,z_new,a_old]!=0)
            x2_new[:,:,z_new,a_old][sigma2_0] = x2_new_tep[sigma2_0] /sigma2[:,:,z_new,a_old][sigma2_0] 

      belief1_f = np.floor(x1_new*(num_discrete-1))/(num_discrete-1)
      belief1_c = np.ceil(x1_new*(num_discrete-1))/(num_discrete-1)
      iterpolate1 = np.zeros(belief1_f.shape)
      iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (x1_new-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]

      belief2_f = np.floor(x2_new*(num_discrete-1))/(num_discrete-1)
      belief2_c = np.ceil(x2_new*(num_discrete-1))/(num_discrete-1)
      iterpolate2 = np.zeros(belief2_f.shape)
      iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (x2_new-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]
  elif T==1: #generate 1 period data for data generation
      x1_new = np.zeros(x1_old.shape)
      x2_new = np.zeros(x2_old.shape)

      sigma1 = np.zeros(x1_old.shape)
      sigma2 = np.zeros(x2_old.shape)

      # #x_old = np.vstack([x_old,1-x_old]).T
      # za = za_old%(agent+1)
      a_old = a1_old * 2 + a2_old
     
      x1_new_tep = (x1_old  *trans[0,0,a_old] +(1-x1_old)*trans[1,0,a_old])* observe1[0,za_old,z_old]
      sigma1 = x1_new_tep + (x1_old *trans[0,1,a_old] + (1-x1_old)*trans[1,1,a_old] )* observe1[1,za_old,z_old]
      x1_new = x1_new_tep/sigma1

      x2_new_tep = (x2_old  *trans[0,0,a_old] +(1-x2_old)*trans[1,0,a_old])* observe2[0,za_old,z_old]
      sigma2 = x2_new_tep + (x2_old *trans[0,1,a_old] + (1-x2_old)*trans[1,1,a_old] )* observe2[1,za_old,z_old]
      x2_new = x2_new_tep/sigma2

      belief1_f = np.floor(x1_new*(num_discrete-1))/(num_discrete-1)
      belief1_c = np.ceil(x1_new*(num_discrete-1))/(num_discrete-1)
      iterpolate1 = np.zeros(belief1_f.shape)
      iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (x1_new-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]

      belief2_f = np.floor(x2_new*(num_discrete-1))/(num_discrete-1)
      belief2_c = np.ceil(x2_new*(num_discrete-1))/(num_discrete-1)
      iterpolate2 = np.zeros(belief2_f.shape)
      iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (x2_new-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]

  else:      #generate whole belief for recover process
      x1_new = np.zeros([T,x1_old.shape[0]]) # T, N
      sigma1 = np.zeros([T-1,x1_old.shape[0]]) # T-1 ,N
      x1_new[0,:] = x1_old

      x2_new = np.zeros([T,x2_old.shape[0]]) # T, N
      sigma2 = np.zeros([T-1,x2_old.shape[0]]) # T-1 ,N
      x2_new[0,:] = x2_old
      #x_old = np.vstack([x_old,1-x_old]).T

      for t in range(1,T):
        a1 = a1_old[t-1,:]
        a2 = a2_old[t-1,:]
        a_old = a1 *2 + a2
        za = z_old[t-1,:]
        z = z_old[t,:]
        x1_new_tep = (x1_new[t-1,:] *trans[0,0,a_old] + (1-x1_new[t-1,:])*trans[1,0,a_old] )* observe1[0,za,z]
        sigma1[t-1,:] = x1_new_tep + (x1_new[t-1,:] *trans[0,1,a_old] + (1-x1_new[t-1,:])*trans[1,1,a_old] )* observe1[1,za,z]
        x1_new[t,:] = x1_new_tep/sigma1[t-1,:]
        
        x2_new_tep = (x2_new[t-1,:] *trans[0,0,a_old] + (1-x2_new[t-1,:])*trans[1,0,a_old] )* observe2[0,za,z]
        sigma2[t-1,:] = x2_new_tep + (x2_new[t-1,:] *trans[0,1,a_old] + (1-x2_new[t-1,:])*trans[1,1,a_old] )* observe2[1,za,z]
        x2_new[t,:] = x2_new_tep/sigma2[t-1,:]

      belief1_f = np.floor(x1_new*(num_discrete-1))/(num_discrete-1)
      belief1_c = np.ceil(x1_new*(num_discrete-1))/(num_discrete-1)
      iterpolate1 = np.zeros(belief1_f.shape)
      iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (x1_new-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]      

      belief2_f = np.floor(x2_new*(num_discrete-1))/(num_discrete-1)
      belief2_c = np.ceil(x2_new*(num_discrete-1))/(num_discrete-1)
      iterpolate2 = np.zeros(belief2_f.shape)
      iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (x2_new-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]      

  return sigma1,sigma2,x1_new,x2_new,[iterpolate1,np.int_(belief1_f*(num_discrete-1)),np.int_(belief1_c*(num_discrete-1))],[iterpolate2,np.int_(belief2_f*(num_discrete-1)),np.int_(belief2_c*(num_discrete-1))]
#print(SigmaLambda(None,z_old,x_old,x_old,trans,observe1,observe2,a1_old,a2_old)[0][:,0,1,0])

# trans,observe1,observe2 = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# sigma1,sigma2,x1_new,x2_new,xiterpolate1,xiterpolate2 = SigmaLambda(None,z_old,x_old,x_old,trans,observe1,observe2,a1_old,a2_old)

def ValueIteration(theta1,sigma1,sigma2,xiterpolate1,xiterpolate2, num_discrete=num_discrete,beta = 0.85,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt,za, a1 or xt,za, a2
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #st,z*a_{t-1},a_{t,1},a_{t,2}
  iterpolate1,belief1_f,belief1_c = xiterpolate1[0],xiterpolate1[1],xiterpolate1[2]#xt,zt,z_{t+1}
  iterpolate2,belief2_f,belief2_c = xiterpolate2[0],xiterpolate2[1],xiterpolate2[2]#xt,zt,z_{t+1}

  x1_old = np.linspace(0,1,num=num_discrete).reshape([-1,1])
  x2_old = np.linspace(0,1,num=num_discrete).reshape([-1,1])

  #x_old = np.vstack([x_old,1-x_old]).T

  pi1_new = 0.5*np.ones([num_discrete,num_discrete,za_dim,2]) #xt1,xt2,za,a1

  pi2_old = pi1_new.copy()
  #count = 0
  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros([num_discrete,num_discrete,za_dim,2]) #xt1,xt2,za,a1
    Q2_new = np.zeros([num_discrete,num_discrete,za_dim,2]) #xt1,xt2,za,a2

    for il in range(1000):
      Q2_old = Q2_new.copy() #xt,za,a2
      za = za_old%(agent+1)

      for a2 in range(2):
        #a1 = 0
        V2_a10 = []
        for z_new in z_old:
          znew_a10= (0 *2+a2)*3 + z_new
          it01,bf01,bc01 = iterpolate1[:,za,z_new],belief1_f[:,za,z_new],belief1_c[:,za,z_new]
          it02,bf02,bc02 = iterpolate2[:,za,z_new],belief2_f[:,za,z_new],belief2_c[:,za,z_new]

          #print(znew_a10,bf0)
          V2_a10.append( np.stack([it0*Q2_old[bf01,bf02,znew_a10,0] + (1-it0)*Q2_old[bc0,znew_a10,0], it0*Q2_old[bf0,znew_a10,1] + (1-it0)*Q2_old[bc0,znew_a10,1] ],axis=2))
        V2_a10 =  np.stack(V2_a10,axis=3)

        Q2_tepa10 = x_old.dot(rw2[0,:,0,a2].reshape([1,-1])) + (1-x_old).dot(rw2[1,:,0,a2].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V2_a10,axis=2)),axis=2)

        #a1 = 1
        V2_a11 = []
        for z_new in z_old:
          znew_a11= (1 *2+a2)*3 + z_new
          it1,bf1,bc1 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V2_a11.append( np.stack([it1*Q2_old[bf1,znew_a11,0] + (1-it1)*Q2_old[bc1,znew_a11,0], it1*Q2_old[bf1,znew_a11,1] + (1-it1)*Q2_old[bc1,znew_a11,1] ],axis=2))
        V2_a11 =  np.stack(V2_a11,axis=3)

        Q2_tepa11 = x_old.dot(rw2[0,:,1,a2].reshape([1,-1])) + (1-x_old).dot(rw2[1,:,1,a2].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V2_a11,axis=2)),axis=2)

          
        Q2_new[:,:,a2] = pi1_old[:,:,0] *Q2_tepa10 + pi1_old[:,:,1] * Q2_tepa11
        #print(it0)
      #print(il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))),np.max(np.abs(Q2_old)))
      if np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
        break

    pi2_new = softmax(Q2_new,axis=2)   
    count = np.max(np.abs(np.nan_to_num((pi2_old-pi2_new)/pi2_old)))


    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()
      za = za_old%(agent+1)

      for a1 in range(2):
        #a2 = 0
        V1_a20 = []
        for z_new in z_old:
          znew_a20= (a1 *2+0)*3 + z_new
          it0,bf0,bc0 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V1_a20.append( np.stack([it0*Q1_old[bf0,znew_a20,0] + (1-it0)*Q1_old[bc0,znew_a20,0], it0*Q1_old[bf0,znew_a20,1] + (1-it0)*Q1_old[bc0,znew_a20,1] ],axis=2))
        V1_a20 =  np.stack(V1_a20,axis=3)

        Q1_tepa20 = x_old.dot(rw1[0,:,a1,0].reshape([1,-1])) + (1-x_old).dot(rw1[1,:,a1,0].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V1_a20,axis=2)),axis=2)

        #a2 = 1
        V1_a21 = []
        for z_new in z_old:
          znew_a21= (a1 *2+1)*3 + z_new
          it1,bf1,bc1 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V1_a21.append( np.stack([it1*Q1_old[bf1,znew_a21,0] + (1-it1)*Q1_old[bc1,znew_a21,0], it1*Q1_old[bf1,znew_a21,1] + (1-it1)*Q1_old[bc1,znew_a21,1] ],axis=2))
        V1_a21 =  np.stack(V1_a21,axis=3)

        Q1_tepa21 = x_old.dot(rw1[0,:,a1,1].reshape([1,-1])) + (1-x_old).dot(rw1[1,:,a1,1].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V1_a21,axis=2)),axis=2)

          
        Q1_new[:,:,a1] = pi2_old[:,:,0] *Q1_tepa20 + pi2_old[:,:,1] * Q1_tepa21

      #print(il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))),np.max(np.abs(Q1_old)))

      if np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))) <thread_inner:
        #print('Q1',il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))
        break

    pi1_new = softmax(Q1_new,axis=2)
    
    #print('pi1 pi2',ol,np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))),count)

    if np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

trans,observe1,observe2 = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
sigma1,sigma2,x1_new,x2_new,xiterpolate1,xiterpolate2 = SigmaLambda(None,z_old,x_old,x_old,trans,observe1,observe2,a1_old,a2_old)
# Q1,Q2,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)

#2. Data Generation Process

In [ ]:
trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
trans-np.diag(np.ones(2))

array([[-0.1,  0.1],
       [ 0.1, -0.1]])

In [ ]:
int(np.sqrt(SamplePath/SubSample))

20

In [ ]:
##################################Global Parameter#################################
# theta1 = np.array([3,1.9,2]) #RS, FC, EC
# theta23 = 0.2
# agent = 2
# za_dim =(agent+1)*(2**agent)
# za_old = np.arange(z_dim,dtype = int)
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
###################################################################################
from tqdm import tqdm 

def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath,N_sub = SubSample):
  # Assume start from good condition and highest market size(2), agents entry market(1) at time 0

  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)#xt,za, a1 or xt,za, a2


  condition = np.zeros([T,N],dtype= int)
  condition[0,:] = np.random.choice(np.array(2,dtype=int),size=N,p=[1/2,1/2])

  market_size = np.zeros([T,N],dtype = int) #observation state
  market_size[0,:] = np.random.choice(np.array(3,dtype=int),size=N,p=[1/3,1/3,1/3])

  belief1 = np.zeros([T,N])
  haha = int(np.sqrt(N/N_sub))
  belief1[0,:] = np.kron(np.kron(np.linspace(0,1,num=haha),np.ones(haha)),np.ones(N_sub))#np.kron(np.linspace(0,1,num=int(N/N_sub)),np.ones(N_sub))
  belief2 = np.zeros([T,N])
  belief2[0,:] = np.kron(np.kron(np.ones(haha),np.linspace(0,1,num=haha)),np.ones(N_sub))#np.kron(np.linspace(0,1,num=int(N/N_sub)),np.ones(N_sub))
  
  sigma_val1 = np.ones([T-1,N])
  sigma_val2 = np.ones([T-1,N])

  belief1_f = np.floor(belief1[0,:]*(num_discrete-1))/(num_discrete-1)
  belief1_c = np.ceil(belief1[0,:]*(num_discrete-1))/(num_discrete-1)  
  belief1_f,belief1_c = np.int_(belief1_f*(num_discrete-1)),np.int_(belief1_c*(num_discrete-1))

  belief2_f = np.floor(belief2[0,:]*(num_discrete-1))/(num_discrete-1)
  belief2_c = np.ceil(belief2[0,:]*(num_discrete-1))/(num_discrete-1)   
  belief2_f,belief2_c = np.int_(belief2_f*(num_discrete-1)),np.int_(belief2_c*(num_discrete-1))

  action1 = np.ones([T,N],dtype = int)   #agent1's action
  action2 = np.ones([T,N],dtype = int)   #agent2's action
  # za = (np.zeros(N,dtype=int) *2 + np.zeros(N,dtype=int))*3+market_size[0,:]
  # action_mat1 = np.random.random(size=N)
  # p1 = iterpolate * pi1[belief_f,za,0] + (1-iterpolate) * pi1[belief_c,za,0]
  # action1[0,:] = np.int_(p1<=action_mat1)
  
  # action_mat2 = np.random.random(size=N)
  # p2 = iterpolate * pi2[belief_f,za,0] + (1-iterpolate) * pi2[belief_c,za,0]
  # action2[0,:] = np.int_(p2<=action_mat2)


  for t in tqdm(range(1,T)):
    #generate new condition #s_{t+1}
    cd_mat = np.random.random(size=N)
    pc = trans[condition[t-1,:],0]
    condition[t,:] = np.int_(pc<=cd_mat)
   
    # generate new market size #z_{t+1}
    ms_mat = np.random.random(size=N)
    pt = observe[condition[t,:],market_size[t-1,:],:]
    market_size[t,:] = np.int_(pt[:,0]<=ms_mat) +np.int_(pt[:,0]+pt[:,1]<=ms_mat)

    # update new belief #x_t
    za = market_size[t-1,:]
    z = market_size[t,:]
    x_new_tep1 = (belief1[t-1,:] *trans[0,0] + (1-belief1[t-1,:])*trans[1,0] )* observe[0,za,z]
    x_new_tep2 = (belief2[t-1,:] *trans[0,0] + (1-belief2[t-1,:])*trans[1,0] )* observe[0,za,z]

    #print(x_new_tep)
    sigma_val1[t-1,:] = x_new_tep1 + (belief1[t-1,:] *trans[0,1] + (1-belief1[t-1,:])*trans[1,1] )* observe[1,za,z]
    sigma_val2[t-1,:] = x_new_tep2 + (belief2[t-1,:] *trans[0,1] + (1-belief2[t-1,:])*trans[1,1] )* observe[1,za,z]

    belief1[t,:] = x_new_tep1/sigma_val1[t-1,:]
    belief2[t,:] = x_new_tep2/sigma_val2[t-1,:]

    belief1_f = np.floor(belief1[t,:]*(num_discrete-1))/(num_discrete-1)
    belief1_c = np.ceil(belief1[t,:]*(num_discrete-1))/(num_discrete-1)
    iterpolate1 = np.zeros(belief1_f.shape)
    iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (belief1[t,:]-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]      
    belief1_f,belief1_c = np.int_(belief1_f*(num_discrete-1)),np.int_(belief1_c*(num_discrete-1))

    belief2_f = np.floor(belief2[t,:]*(num_discrete-1))/(num_discrete-1)
    belief2_c = np.ceil(belief2[t,:]*(num_discrete-1))/(num_discrete-1)
    iterpolate2 = np.zeros(belief2_f.shape)
    iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (belief2[t,:]-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]      
    belief2_f,belief2_c = np.int_(belief2_f*(num_discrete-1)),np.int_(belief2_c*(num_discrete-1))

    # generate new action for both agents #a_{t+1}
    za = (action1[t-1,:] *2 + action2[t-1,:])*3+market_size[t,:]
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate1 * pi1[belief1_f,za,0] + (1-iterpolate1) * pi1[belief1_c,za,0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = iterpolate2 * pi2[belief2_f,za,0] + (1-iterpolate2) * pi2[belief2_c,za,0]
    action2[t,:] = np.int_(p2<=action_mat2)

  return market_size, action1,action2 ,belief1,belief2,sigma_val1,sigma_val2,condition


market_size, action1,action2 ,belief1,belief2,sigma_val1,sigma_val2,condition = DataGeneration()
#haha = SigmaLambda()

100%|██████████| 99/99 [00:00<00:00, 927.82it/s]


In [ ]:
np.sum(condition)/np.prod(condition.shape)

0.5021975

#3.1 Recover Dynamic Part (Partial)

In [ ]:
theta23

[0.9, 0.1, 0.2, 0.3]

In [ ]:
from scipy.optimize import minimize
ini_x0 = 0.9*np.array(theta23)
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod ,x10 = belief1[0,:],x20 = belief2[0,:]):
  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma_val1,_,_ = SigmaLambda(None,market_size,x10,trans,observe,T=T)
  sigma_val2,_,_ = SigmaLambda(None,market_size,x20,trans,observe,T=T)

  # sigma_val,_,_ = SigmaLambda(None,market_size,x_old,action1,action2,theta23= theta23,num_discrete=num_discrete,T = T)
  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(sigma_val1))-np.sum(np.log(sigma_val2))
  print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.49999],[0.00001,0.49999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: [0.81 0.09 0.18 0.27]
[0.81 0.09 0.18 0.27] 569641.6356878079
[0.81000001 0.09       0.18       0.27      ] 569641.6357364457
[0.81       0.09000001 0.18       0.27      ] 569641.6357970922
[0.81       0.09       0.18000001 0.27      ] 569641.6355356133
[0.81       0.09       0.18       0.27000001] 569641.6352861791
[1.0000e-05 1.0000e-05 4.9999e-01 4.9999e-01] 1880642.1042736059
[1.0010e-05 1.0000e-05 4.9999e-01 4.9999e-01] 1880642.1042736059
[1.0000e-05 1.0010e-05 4.9999e-01 4.9999e-01] 1880642.1042736059
[1.0000000e-05 1.0000000e-05 4.9999001e-01 4.9999000e-01] 1880642.1055886538
[1.0000000e-05 1.0000000e-05 4.9999000e-01 4.9999001e-01] 1880773.6734584211
[0.53722348 0.05969449 0.28776153 0.34745265] 598442.0578070803
[0.53722349 0.05969449 0.28776153 0.34745265] 598442.0577022901
[0.53722348 0.0596945  0.28776153 0.34745265] 598442.0571364597
[0.53722348 0.05969449 0.28776154 0.34745265] 598442.0586707382
[0.53722348 0.05969449 0.28776153 0.34745266] 598442.0634313636
[

KeyboardInterrupt: ignored

#3.2 Recover Reward Part (Partial)

In [ ]:
theta1.reshape(-1)

array([ 1.5,  1. , -1.9, -1.8,  1. ])

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val1,_ = SigmaLambda(None,market_size,belief1[0,:],trans,observe,T=TimePeriod)
_,belief_val2,_ = SigmaLambda(None,market_size,belief2[0,:],trans,observe,T=TimePeriod)

sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val1=belief_val1,belief_val2=belief_val2,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief1_f = np.floor(belief_val1[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief1_c = np.ceil(belief_val1[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate1 = np.zeros(belief1_f.shape)
  iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (belief_val1[1:T,:]-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]      
  belief1_f,belief1_c = np.int_(belief1_f*(num_discrete-1)),np.int_(belief1_c*(num_discrete-1))


  belief2_f = np.floor(belief_val2[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief2_c = np.ceil(belief_val2[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate2 = np.zeros(belief2_f.shape)
  iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (belief_val2[1:T,:]-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]      
  belief2_f,belief2_c = np.int_(belief2_f*(num_discrete-1)),np.int_(belief2_c*(num_discrete-1))


  p1 = iterpolate1*pi1[belief1_f,za,action1[1:T,:]] +(1- iterpolate1)*pi1[belief1_c,za,action1[1:T,:]]
  p2 = iterpolate2*pi2[belief2_f,za,action2[1:T,:]]+(1- iterpolate2)*pi2[belief2_c,za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 193896.88124080774
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 193896.88122636848
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 193896.88122625975
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 193896.88136377663
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 193896.88137256197
[ 1.275       0.85       -1.615      -1.53        0.85000001] 193896.88108385744
[ 100.  100. -100. -100.  100.] 9377954.444444444
[ 100.00000001  100.         -100.         -100.          100.        ] 9377954.444469959
[ 100.          100.00000001 -100.         -100.          100.        ] 9377954.444470264
[ 100.          100.          -99.99999999 -100.          100.        ] 9377954.444197383
[ 100.          100.         -100.          -99.99999999  100.        ] 9377954.444170244
[ 100.          100.         -100.         -100.          100.00000001] 9377954.

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(res1.x)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val1,_ = SigmaLambda(None,market_size,belief1[0,:],trans,observe,T=TimePeriod)
_,belief_val2,_ = SigmaLambda(None,market_size,belief2[0,:],trans,observe,T=TimePeriod)

sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val1=belief_val1,belief_val2=belief_val2,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief1_f = np.floor(belief_val1[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief1_c = np.ceil(belief_val1[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate1 = np.zeros(belief1_f.shape)
  iterpolate1[np.where((belief1_f-belief1_c)!=0)] = (belief_val1[1:T,:]-belief1_c)[np.where((belief1_f-belief1_c)!=0)]/(belief1_f-belief1_c)[np.where((belief1_f-belief1_c)!=0)]      
  belief1_f,belief1_c = np.int_(belief1_f*(num_discrete-1)),np.int_(belief1_c*(num_discrete-1))


  belief2_f = np.floor(belief_val2[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief2_c = np.ceil(belief_val2[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate2 = np.zeros(belief2_f.shape)
  iterpolate2[np.where((belief2_f-belief2_c)!=0)] = (belief_val2[1:T,:]-belief2_c)[np.where((belief2_f-belief2_c)!=0)]/(belief2_f-belief2_c)[np.where((belief2_f-belief2_c)!=0)]      
  belief2_f,belief2_c = np.int_(belief2_f*(num_discrete-1)),np.int_(belief2_c*(num_discrete-1))


  p1 = iterpolate1*pi1[belief1_f,za,action1[1:T,:]] +(1- iterpolate1)*pi1[belief1_c,za,action1[1:T,:]]
  p2 = iterpolate2*pi2[belief2_f,za,action2[1:T,:]]+(1- iterpolate2)*pi2[belief2_c,za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 193861.05316465744
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 193861.0531494866
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 193861.05315106746
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 193861.05328708913
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 193861.05329582503
[ 1.275       0.85       -1.615      -1.53        0.85000001] 193861.05300833366
[ 100.  100. -100. -100.  100.] 9377954.444444444
[ 100.00000001  100.         -100.         -100.          100.        ] 9377954.444471452
[ 100.          100.00000001 -100.         -100.          100.        ] 9377954.444468772
[ 100.          100.          -99.99999999 -100.          100.        ] 9377954.444197383
[ 100.          100.         -100.          -99.99999999  100.        ] 9377954.444170244
[ 100.          100.         -100.         -100.          100.00000001] 9377954.4

#4.1 Recover Dynamic Part (Complete)

In [ ]:
def Dynamic(theta23):
  
  observe = np.zeros([3,3]) #z_t,z_{t+1}
  observe[0,1] = observe[1,0] = observe[1,2]= observe[2,1] = theta23
  observe[0,0] = 1 - observe[0,1] -observe[0,2]
  observe[1,1] = 1 - observe[1,0] -observe[1,2]
  observe[2,2] = 1 - observe[2,0] - observe[2,1]

  return observe
#print(np.kron(np.ones(4),Dynamic(theta23)).T)

def Reward(theta1,ag_i): # 3*2*2, 2,2
  rw = np.zeros([(agent+1)*(2**agent),2,2])#z*a_{t-1},a_{t,1},a_{t,2}
  if ag_i==1:
    rw[:,1,0] = (za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[1] - theta1[3] * (1-((za_old//(agent+1))//2))
    rw[:,1,1] = (za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[1] - theta1[3] * (1-((za_old//(agent+1))//2))
  elif ag_i==2:
    rw[:,0,1] = (za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[2] - theta1[3] * (1-((za_old//(agent+1))%2))
    rw[:,1,1] = (za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[2] - theta1[3] * (1-((za_old//(agent+1))%2))    

  return rw
#print(Reward(theta1,2))

def ValueIteration(theta1,theta23,beta = 0.85,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5,max_iter = 50000): #zt * a_old, a1 or zt * a_old,a2
  observe = Dynamic(theta23)
  observe = np.kron(np.ones(4),observe).T
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #z*a_{t-1},a_{t,1},a_{t,2}

  pi1_new = 0.5*np.ones([za_dim,2])#zt*a_old, a1
  pi2_old = pi1_new.copy()

  for ol in range(max_iter):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros([za_dim,2]) #xt,zt,a_old, a1
    Q2_new = np.zeros([za_dim,2])

    for il in range(max_iter):
      Q2_old = Q2_new.copy()
      for a2 in range(2):
        #a1=0
        znew_a10= (0 *2+a2)*3 * np.ones([za_dim,1],dtype = int)+z_old
        #print(znew_a10) 
        V2_a10 = Q2_old[znew_a10,:]
        Q2_tepa10 = rw2[:,0,a2] + beta * np.sum(observe*(gamma+logsumexp(V2_a10,axis=2)),axis=1)

        #a1=1
        znew_a11= (1 *2+a2)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V2_a11 =  Q2_old[znew_a11,:]
        Q2_tepa11 =rw2[:,1,a2] +  beta * np.sum(observe*(gamma+logsumexp(V2_a11,axis=2)),axis=1)
          
        Q2_new[:,a2] = pi1_old[:,0] *Q2_tepa10 + pi1_old[:,1]* Q2_tepa11
      #print(il,np.max(np.abs(((Q2_new-Q2_old)/Q2_old))))
      if np.max(np.abs(((Q2_new-Q2_old)/Q2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)))
        break

    pi2_new = softmax(Q2_new,axis=1)
    count = np.max(np.abs(((pi2_old-pi2_new)/pi2_old))) 

    pi2_old = pi2_new.copy()
    for il in range(max_iter):
      Q1_old = Q1_new.copy()
      for a1 in range(2):
        #a2 = 0
        znew_a20= (a1 *2+0)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V1_a20 =  Q1_old[znew_a20,:]
        Q1_tepa20 =rw1[:,a1,0] + beta * np.sum(observe*(gamma+logsumexp(V1_a20,axis=2)),axis=1)

        #a2=1
        znew_a21= (a1 *2+1)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V1_a21 =  Q1_old[znew_a21,:]
        Q1_tepa21 = rw1[:,a1,1] + beta * np.sum(observe*(gamma+logsumexp(V1_a21,axis=2)),axis=1)
          
        Q1_new[:,a1] = pi2_old[:,0] *Q1_tepa20 + pi2_old[:,1] * Q1_tepa21
      #print(il,np.max(np.abs(((Q1_new-Q1_old)/Q1_old))))
      if np.max(np.abs(((Q1_new-Q1_old)/Q1_old))) <thread_inner:
        #print('Q1',il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
        break

    pi1_new = softmax(Q1_new,axis=1)
    
    #print('pi1 pi2',ol,np.max(np.abs(((pi1_old-pi1_new)/pi1_old))),count)
    if np.max(np.abs(((pi1_old-pi1_new)/pi1_old))) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

In [ ]:
theta23[2:]

[0.2, 0.3]

In [ ]:
from scipy.optimize import minimize
ini_x0 = 0.85*np.average(theta23[2:])
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod):
  observe = Dynamic(theta23)

  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(observe[market_size[0:-1,:],market_size[1:T,:]]))
  #print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0,0.499999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: 0.2125
      fun: 286148.03075737815
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 14
      nit: 4
   status: 0
  success: True
        x: array([0.25022829])


#4.2 Recover Reward Part (Complete)

In [ ]:
theta1.reshape(-1)

array([ 1.5,  1. , -1.9, -1.8,  1. ])

In [ ]:
ini_x0 = 0.85*(np.hstack([np.sum(theta1[0:2]),theta1[2:]])).reshape(-1)
print('start from:',ini_x0 )

def func2(theta1,theta23=res1.x,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod):
  #theta1 = theta1.reshape([2,2,2])
  _,_,pi1,pi2 = ValueIteration(theta1,theta23)

  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]

  p1 = pi1[za,action1[1:T,:]]
  p2 = pi2[za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res

res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 2.125 -1.615 -1.53   0.85 ]
[ 2.125 -1.615 -1.53   0.85 ] 191746.13531765185
[ 2.12500001 -1.615      -1.53        0.85      ] 191746.1353052428
[ 2.125      -1.61499999 -1.53        0.85      ] 191746.13540031755
[ 2.125      -1.615      -1.52999999  0.85      ] 191746.13540528365
[ 2.125      -1.615      -1.53        0.85000001] 191746.1352093083
[ 100. -100. -100.  100.] 9377954.444444448
[ 100.00000001 -100.         -100.          100.        ] 9377954.444495775
[ 100.          -99.99999999 -100.          100.        ] 9377954.444197385
[ 100.         -100.          -99.99999999  100.        ] 9377954.444170246
[ 100.         -100.         -100.          100.00000001] 9377954.444809645
[ 8.25648143 -7.77843091 -7.69875583  7.06135514] 703546.0066963669
[ 8.25648144 -7.77843091 -7.69875583  7.06135514] 703546.0067476986
[ 8.25648143 -7.7784309  -7.69875583  7.06135514] 703546.0064493362
[ 8.25648143 -7.77843091 -7.69875582  7.06135514] 703546.0064222033
[ 8.25648143 -7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
res = {}
res['MarketSize'] = market_size
res['Action1'] = action1
res['Action2'] = action2 
res['Belief1'] = belief1
res['Belief2'] = belief2
res['Sigma1'] = sigma_val1
res['Sigma2'] = sigma_val2
res['Condition'] = condition
import pickle
# with open('/content/drive/MyDrive/2020_WINTER/Rust/data_final/table3_rust_simulation.txt','rb') as fp:
#   res = pickle.load(fp)
# print(res)
with open('/content/drive/MyDrive/2020_WINTER/Game/Data/simulation.txt',"wb") as fp:
  pickle.dump(res,fp)